In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Correlation Power Analysis

## A better version of password check

```c
uint8_t check_password(uint8_t cmd, uint8_t scmd, uint8_t input_length, uint8_t *input)
{
    trigger_high();

    uint8_t password_wrong = 0;
    for (unsigned int i = 0; i < sizeof(stored_password) - 1; i++)
    {
        uint8_t stored = stored_password[i];
        uint8_t attempt = input[i];
        password_wrong = stored ^ attempt;
    }

    trigger_low();

    simpleserial_put(0x01, 1, &password_wrong);
    return 0;
}
```

### Improvements

😎 No timing dependency.

😎 No data dependent code flow.

## Try again: Recording different attempts

In [ ]:
import securec
import securec.util
scope, target = securec.util.init()
scope.default_setup()

In [ ]:
securec.util.compile_and_flash('./example_3.c')

In [ ]:
scope.default_setup()
securec.util.reset_target()
def capture(attempt, samples=500):
    scope.adc.samples = samples
    scope.arm()
    if isinstance(attempt, int):
        attempt = bytes([attempt])
    elif isinstance(attempt, list):
        attempt = bytes(attempt)
    elif isinstance(attempt, str):
        attempt = attempt.encode()
    target.simpleserial_write(0x01, attempt + (8 - len(attempt)) * b'\x00')
    trace = securec.util.capture()
    return trace, not bool(target.simpleserial_read(0x01)[0])

In [ ]:
import math
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label, LinearColorMapper, Span
from bokeh.palettes import Category10_10

output_notebook()

In [ ]:
def plot_attempts(
    attempts,
    shift_y=True,
    max_x=None,
):
    data = []
    for attempt in attempts:
        data.append(capture(attempt))
    p = figure(height=300, sizing_mode='stretch_width')
    p.add_tools(CrosshairTool())
    for idx, (attempt, (trace, result)) in enumerate(zip(attempts, data)):
        if max_x:
            trace = trace[:max_x]
        p.line(
            range(0, len(trace)), 
            trace - (idx * 0.6 if shift_y else 0), 
            line_color=Category10_10[idx], 
            legend_label=f'{attempt} -> {result}'
        )
    show(p)

In [ ]:
scope.default_setup()
securec.util.reset_target()
plot_attempts(['hello', 'world', 'ifx'])

## Concentrate on small differences

In [ ]:
plot_attempts(['hello', 'world', 'ifx'], shift_y=False, max_x=100)

### Where do these differences come from?

Main principle of power analysis:

> The power consumption of a device depends on the data it processes.

More precise:

> The power consumption of a device is proportional to the hamming weight of the data it processes.

## "Viewing" hamming weights

In [ ]:
plot_attempts([b'\x00', b'\xff'], shift_y=False, max_x=80)

### Link differences with code

#### Assembly of `check_password()`

```asm
uint8_t check_password(uint8_t cmd, uint8_t scmd, uint8_t input_length, uint8_t *input)
{
 21a:	cf 93       	push	r28
 21c:	df 93       	push	r29
 21e:	1f 92       	push	r1
 220:	cd b7       	in	r28, 0x3d	; 61
 222:	de b7       	in	r29, 0x3e	; 62
    trigger_high();
 224:	81 e0       	ldi	r24, 0x01	; 1
 226:	80 93 05 06 	sts	0x0605, r24	; 0x800605 <__TEXT_REGION_LENGTH__+0x7de605>
 22a:	a0 e0       	ldi	r26, 0x00	; 0
 22c:	b0 e2       	ldi	r27, 0x20	; 32
 22e:	f9 01       	movw	r30, r18
 230:	80 e0       	ldi	r24, 0x00	; 0

    uint8_t password_wrong = 0;
    for (unsigned int i = 0; i < sizeof(stored_password) - 1; i++)
    {
        uint8_t stored = stored_password[i];
 232:	2d 91       	ld	r18, X+
        uint8_t attempt = input[i];
 234:	91 91       	ld	r25, Z+
        password_wrong |= stored ^ attempt;
 236:	92 27       	eor	r25, r18
 238:	89 2b       	or	r24, r25
 23a:	90 e2       	ldi	r25, 0x20	; 32
 23c:	a8 30       	cpi	r26, 0x08	; 8
 23e:	b9 07       	cpc	r27, r25
 240:	c1 f7       	brne	.-16     	; 0x232 <check_password+0x18>
 242:	89 83       	std	Y+1, r24	; 0x01
```

### View hamming weight of input

In [ ]:
def plot_attempts_with_code(
    attempts,
    x_range=(20, 60),
):
    data = []
    for attempt in attempts:
        data.append(capture(attempt))
    p = figure(height=300, sizing_mode='stretch_width', x_range=x_range)
    p.add_tools(CrosshairTool())
    for idx, (attempt, (trace, _)) in enumerate(zip(attempts, data)):
        p.line(
            range(0, len(trace)), 
            trace, 
            line_color=Category10_10[idx], 
            legend_label=f'{attempt}'
        )

    for x, label in zip(
        range(28, 80, 4), 
        ('ld r18, X+', '', 'ld r25, Z+', '', 'eor r25, r18', 'or r24, r25')
    ):
        if not label:
            continue
        p.add_layout(Span(location=x, dimension='height', line_color='darkslateblue', line_width=30, line_alpha=0.1))
        p.add_layout(Label(x=x, y=p.plot_height, text=label, y_units='screen', x_offset=-15, y_offset=-35,
                        text_align='right', text_color='darkslateblue', angle=math.pi/2))
        for idx, (trace, _) in enumerate(data):
            p.circle(x, trace[x], size=10, color=Category10_10[idx])

    show(p)

In [ ]:
plot_attempts_with_code([b'\x00', b'\x0F', b'\xff'])

## Make proportionality measurable: Pearson Correlation Coefficient

For two random variables $X, Y$ the *Pearson correlation coefficient (PCC)* is defined as

$$\rho_{X,Y} := \frac{\mathrm{Cov}(X, Y)}{\sqrt{\mathrm{Var}(X)} \sqrt{\mathrm{Var}(Y)}} \ \in [-1, 1]\,.$$

For two samples of finite length $x = {x_1, ..., x_n}$, $y = {y_1, ..., y_n}$ it can be defined as 

$$r_{x,y} := \frac{\sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)}{\sqrt{\sum_{i=1}^n (x_i - \bar x)^2}\sqrt{\sum_{i=1}^n (y_i - \bar y)^2}} \ \in [-1, 1]\,,$$

where $\bar x := \frac{1}{n} \sum_{i=1}^n x_i$ is the mean of a sample $x$.

In [ ]:
import numpy as np

def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [ ]:
HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

### Plot correlation against input

In [ ]:
def plot_attempts_with_code_and_pearson(
    attempts,
    x_range=(20, 60),
):
    traces = []
    for attempt in attempts:
        traces.append(capture(attempt)[0])
    traces = np.array(traces)
    p = figure(height=600, sizing_mode='stretch_width', x_range=x_range)
    p.add_tools(CrosshairTool())
    for idx, (attempt, trace) in enumerate(zip(attempts, traces)):
        p.line(
            range(0, len(trace)), 
            trace, 
            line_color=Category10_10[idx % 10], 
            legend_label=f'{attempt}'
        )

    for x, label in zip(
        range(28, 80, 4), 
        ('ld r18, X+', '', 'ld r25, Z+', '', 'eor r25, r18', 'or r24, r25')
    ):
        if not label:
            continue
        p.add_layout(Span(location=x, dimension='height', line_color='darkslateblue', line_width=30, line_alpha=0.1))
        p.add_layout(Label(x=x, y=p.plot_height, text=label, y_units='screen', x_offset=-15, y_offset=-35,
                        text_align='right', text_color='darkslateblue', angle=math.pi/2))
        p.add_layout(Label(x=x, y=50, text=f'pcc(hw(attempt), trace[:, {x}]): {pearson([hw(a[0]) for a in attempts], traces[:, x]):.2f}', y_units='screen', x_offset=-15, y_offset=-35,
                        text_align='left', text_color='darkslateblue', angle=math.pi/2))
        for idx, trace in enumerate(traces):
            p.circle(x, trace[x], size=10, color=Category10_10[idx % 10])

    show(p)


In [ ]:
def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [ ]:
import random
import tqdm
import tqdm.notebook

def capture_random(trace_samples=500, trace_nums=100):
    traces = []
    attempts = []
    for _ in tqdm.notebook.tqdm(range(trace_nums)):
        attempt = bytes([random.randint(0, 255) for _ in range(10)])
        traces.append(capture(attempt, samples=trace_samples)[0])
        attempts.append(attempt)
    traces = np.array(traces)
    attempts = np.array([list(a) for a in attempts])
    return traces, attempts

In [ ]:
def plot_pearson(traces, attempts, index=0):
    pearsons = abs(pearson_pointwise(traces, hw_vec([a[0] for a in attempts])))
    p = figure(height=300, sizing_mode='stretch_width')
    p.title = "PCC(hw(attempts), traces[:, i])"
    p.add_tools(CrosshairTool())
    p.line(range(0, len(pearsons)), pearsons)
    show(p)

In [ ]:
plot_attempts_with_code_and_pearson([b'\x00', b'\x01', b'\x03', b'\x07', b'\x0F', b'\x1F', b'\x3F', b'\x7F', b'\xFF'])

What happens exactly?

1. Define inputs and capture traces

In [ ]:
attempts = [bytes([i]) for i in range(0, 256, 5)]
traces = np.array([capture(attempt)[0] for attempt in attempts])
print('attempts: ', attempts)
print('traces: ', traces)

2. Calculate hamming weights of attempts

In [ ]:
attempts_hws = [hw(attempt[0]) for attempt in attempts]
print('hamming weights of attempts:', attempts_hws)

3. Calculate PCC of hamming weights and traces at specific point

In [ ]:
pearson(attempts_hws, traces[:, 0])

Calculate PCC for *every* trace point

In [ ]:
plot_pearson(*capture_random())

#### Conclusion

- Pearson correlation coefficient reflects principle that the power consumption of a device is proportional to the hamming weight of the data it processes.
- When calculating `pcc(hw(input), traces[:, i])` for all `i=0..samples` we can "see" where `input` is processed.